In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_data = '//content//drive//My Drive//MLBA_Kaggle_1//train.csv'
test_data = '//content//drive//My Drive//MLBA_Kaggle_1//valid.csv' 

In [ ]:
def getData(path):
#     sequence = []
#     label_str = []
    label = []
    
    with open(train_data) as f:
        sequence = []
        label_str = []
        for line in f:
            text_file = line.split(",");
            if text_file[0] == 'ID':
                continue;
            label_str.append(text_file[1])
            s = text_file[2]
            sequence.append(s[:-1]);
    
    for i in label_str:
        if i == 'DNA':
            label.append(1)
        else:
            label.append(0)
    
    return sequence,label

In [ ]:
x_train,y_train = getData(train_data);

In [ ]:
aminoacid_sequence = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']
dipeptide_seq = aminoacid_sequence

In [ ]:
def get_dipeptite_dictionary(str):
    
    size = len(str)
    
    i = 0
    eachseq_list = []
    while( i < size - 1 ):
        add_seq = str[i] 
        eachseq_list.append(add_seq)
        i += 1
    
        
    
    getdict = {}
    
    for item in eachseq_list:
        if getdict.get(item) == None:
            getdict[item] = 1
        else:
            temp = getdict[item]
            temp += 1
            getdict[item] = temp
    
    new_dict = {}
    for item in getdict:
        value = getdict[item]
        value = value/(len(str))
        new_dict[item] = value
    
    return new_dict

In [ ]:
def getlist_of_dictionary(x_train):
    list_dictionary = []
    
    for each in x_train:
        dic = get_dipeptite_dictionary(each)
        list_dictionary.append(dic)
        
    return list_dictionary

In [ ]:
def getvector(vector_dictionary):
    final_vector = []
    
    for dic in vector_dictionary:
        vec = [0.0] * 20
        for item in dic:
            for seq in dipeptide_seq:
                if seq == item:
                    index = dipeptide_seq.index(seq)
                    vec[index] = dic[item]
                    
        final_vector.append(vec)            
    
    return final_vector

In [ ]:
training_vector_dict = getlist_of_dictionary(x_train)

In [ ]:
X_Train = getvector(training_vector_dict)

In [ ]:
len(X_Train)

3049

In [ ]:
len(y_train)

3049

In [ ]:
x = np.array(X_Train)
y = np.array(y_train)

In [ ]:
x.shape

(3049, 20)

In [ ]:
y.shape

(3049,)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x,y, test_size=0.3, random_state=42)

Test Data

In [ ]:
ID = []
test_sequence = []

with open(test_data) as f:
    for line in f:
        after_split = line.split(",")
        if after_split[0] == 'ID':
            continue;
        ID.append(after_split[0])
        test_sequence.append(after_split[1][:-1])

In [ ]:
len(ID)

1071

In [ ]:
testing_vector_dict = getlist_of_dictionary(test_sequence)

In [ ]:
x_test = getvector(testing_vector_dict)

In [ ]:
x_test = np.array(x_test)

In [ ]:
x_test.shape

(1071, 20)

XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
estimator = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)

In [ ]:
parameters = {
    'max_depth': [4,7],
    'n_estimators': [600],
    'learning_rate': [0.1]
}

In [ ]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'accuracy',
    n_jobs = 10,
    cv = 10,
    verbose=100
)

In [ ]:
grid_search.fit(x_train, y_train)

Fitting 10 folds for each of 2 candidates, totalling 20 fits
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 tasks      | elapsed:   15.4s
[Parallel(n_jobs=10)]: Done   2 out of  20 | elapsed:   15.6s remaining:  2.3min
[Parallel(n_jobs=10)]: Done   3 out of  20 | elapsed:   15.6s remaining:  1.5min
[Parallel(n_jobs=10)]: Done   4 out of  20 | elapsed:   16.5s remaining:  1.1min
[Parallel(n_jobs=10)]: Done   5 out of  20 | elapsed:   16.7s remaining:   50.1s
[Parallel(n_jobs=10)]: Done   6 out of  20 | elapsed:   46.0s remaining:  1.8min
[Parallel(n_jobs=10)]: Done   7 out of  20 | elapsed:   46.4s remaining:  1.4min
[Parallel(n_jobs=10)]: Done   8 out of  20 | elapsed:   46.8s remaining:  1.2min
[Parallel(n_jobs=10)]: Done   9 out of  20 | elapsed:   47.7s remaining:   58.3s
[Parallel(n_jobs=10)]: Done  10 out of  20 | elapsed:   47.8s remaining:   47.8s
[Parallel(n_jobs=10)]: Done  11 out of  20 | elapsed:   47.9s remaining

GridSearchCV(cv=10, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=4, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=42, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=10,
             param_grid={'learning_rate': [0.1], 'max_depth': [4, 7],
                         'n_estimators': [600]},
             pre_dispatch='2*n_jobs', refit=True, return_train_s

In [ ]:
grid_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=None, n_estimators=600, n_jobs=1,
              nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              silent=None, subsample=1, verbosity=1)

In [ ]:
grid_search.best_score_

0.7226207713571146

In [ ]:
YBoostTrain = grid_search.predict(x_train)
YBoostValidate = grid_search.predict(x_val)

In [ ]:
print("\nXGBoost Forest :-")
print("Training Set - AUCCURACY : ",accuracy_score(y_train,YBoostTrain))
print("Validation Set - AUCCURACY : ",accuracy_score(y_val,YBoostValidate))


XGBoost Forest :-
Training Set - AUCCURACY :  1.0
Validation Set - AUCCURACY :  0.7016393442622951


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val,YBoostValidate)

0.6936407766990291

In [ ]:
ans_XGB_gridsearch = grid_search.predict(x_test)

In [ ]:
f = open('XGBoost_20AC_70val.csv','w')

s = "ID,Lable\n"

c = 0
for i in ans_XGB_gridsearch:
    j = -1
    if i == 1:
        s = s+ID[c]+","+i.__str__()+"\n"
    else:
        s = s+ID[c]+","+j.__str__()+"\n"
    c = c + 1

f.write(s)

f.close()

Naive Bayes

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold

cv_method = RepeatedStratifiedKFold(n_splits=5, 
                                    n_repeats=3, 
                                    random_state=999)

In [ ]:
from sklearn.naive_bayes import GaussianNB

np.random.seed(999)

nb_classifier = GaussianNB()

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

gs_NB = GridSearchCV(estimator=nb_classifier, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=100, 
                     scoring='accuracy')

In [ ]:
gs_NB.fit(x_train, y_train);

In [ ]:
gs_NB.best_score_

0.7024412412544482

In [ ]:
accuracy_score(y_val,gs_NB.predict(x_val))

0.6830601092896175

SVM

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import pickle 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [ ]:
svm_model = Pipeline([('clf', SVC())])
param_C = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]
param_scoring = ["accuracy"]

param = [{'clf__C': param_C,'clf__kernel': ['linear','rbf']}]
grid = GridSearchCV(estimator=svm_model,param_grid=param,refit = 'accuracy',scoring=param_scoring,verbose = 100,cv=10,n_jobs=-1)
grid = grid.fit(x_train,y_train)

In [ ]:
print(" The Accuracy of the best model is : ",grid.best_score_)
print(" Best Parameters : ",grid.best_params_)

 The Accuracy of the best model is :  0.7240094774252993
 Best Parameters :  {'clf__C': 1.0, 'clf__kernel': 'rbf'}


In [ ]:
classifier = grid.best_estimator_
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_val)

In [ ]:
print("Accuracy on test data using SVM :",classifier.score(x_val, y_val)*100)

Accuracy on test data using SVM : 71.69398907103826


In [ ]:
ans_SVM = classifier.predict(x_test)

In [ ]:
f = open('SVM_20AC_71val.csv','w')

s = "ID,Lable\n"

c = 0
for i in ans_SVM:
    j = -1
    if i == 1:
        s = s+ID[c]+","+i.__str__()+"\n"
    else:
        s = s+ID[c]+","+j.__str__()+"\n"
    c = c + 1

f.write(s)

f.close()

KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold

cv_method = RepeatedStratifiedKFold(n_splits=5, 
                                    n_repeats=3, 
                                    random_state=999)

In [ ]:
import numpy as np
params_KNN = {'n_neighbors': [1,2,3,4], 
              'p': [1]}

In [ ]:
from sklearn.model_selection import GridSearchCV

gs_KNN = GridSearchCV(estimator=KNeighborsClassifier(), 
                      param_grid=params_KNN, 
                      cv=cv_method,
                      verbose=100,  # verbose: the higher, the more messages
                      scoring='accuracy', 
                      return_train_score=True)

In [ ]:
gs_KNN.fit(x_train, y_train);

Fitting 15 folds for each of 4 candidates, totalling 60 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] n_neighbors=1, p=1 ..............................................
[CV]  n_neighbors=1, p=1, score=(train=1.000, test=0.672), total=   0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[CV] n_neighbors=1, p=1 ..............................................
[CV]  n_neighbors=1, p=1, score=(train=1.000, test=0.642), total=   0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[CV] n_neighbors=1, p=1 ..............................................
[CV]  n_neighbors=1, p=1, score=(train=1.000, test=0.651), total=   0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[CV] n_neighbors=1, p=1 ..............................................
[CV]  n_neighbors=1, p=1, score=(train=1.000, test=0.630), total=   0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 

In [ ]:
gs_KNN.best_params_

{'n_neighbors': 4, 'p': 1}

In [ ]:
gs_KNN.best_score_

0.6850927055960535

In [ ]:
accuracy_score(y_val,gs_KNN.predict(x_val))

0.6775956284153005

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf_rfi = RandomForestClassifier(n_estimators=531,random_state=17)

In [ ]:
clf_rfi.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=531,
                       n_jobs=None, oob_score=False, random_state=17, verbose=0,
                       warm_start=False)

In [ ]:
accuracy_score(y_train,clf_rfi.predict(x_train))

1.0

In [ ]:
accuracy_score(y_val,clf_rfi.predict(x_val))

0.7125683060109289

In [ ]:
ans_RF = clf_rfi.predict(x_test)

In [ ]:
f = open('RF_20AC_71val.csv','w')

s = "ID,Lable\n"

c = 0
for i in ans_RF:
    j = -1
    if i == 1:
        s = s+ID[c]+","+i.__str__()+"\n"
    else:
        s = s+ID[c]+","+j.__str__()+"\n"
    c = c + 1

f.write(s)

f.close()

Ensemble Method

In [ ]:
ans_RF

array([0, 1, 1, ..., 0, 1, 1])

In [ ]:
ans_SVM

array([0, 1, 1, ..., 0, 1, 1])

In [ ]:
ans_XGB_gridsearch

array([0, 1, 1, ..., 0, 1, 1])

In [ ]:
ensemble_rf_svm_xgb = []

In [ ]:
for i in range(len(ans_SVM)):
    
    if ( ans_RF[i] + ans_SVM[i] + ans_XGB_gridsearch[i] ) >= 2:
        ensemble_rf_svm_xgb.append(1)
    else:
        ensemble_rf_svm_xgb.append(0)

In [ ]:
f = open('ensemble_rf_svm_xgb.csv','w')

s = "ID,Lable\n"

c = 0
for i in ensemble_rf_svm_xgb:
    j = -1
    if i == 1:
        s = s+ID[c]+","+i.__str__()+"\n"
    else:
        s = s+ID[c]+","+j.__str__()+"\n"
    c = c + 1

f.write(s)

f.close()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val,grid.predict(x_val))

0.6899227685442987
